In [2]:
import os, sys
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['PYSPARK_PYTHON'] =  'python3.9'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3.9'
os.environ['HADOOP_USER_NAME']='ssenigov'

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [3]:
conf = SparkConf().setAppName('MapType').setMaster('yarn') 
spark = SparkSession.builder.config(conf=conf).getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/26 18:08:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/02/26 18:08:06 WARN DomainSocketFactory: The short-circuit local reads feature cannot be used because libhadoop cannot be loaded.
25/02/26 18:08:06 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [4]:
# from pyspark.sql.functions import create_map, concat_ws, expr, col, lit, \
#    element_at, map_entries, map_from_arrays, array, map_keys, map_values, map_from_entries

In [5]:
from pyspark.sql.functions import create_map

map_col = create_map('k1', 'v1', 'k2', 'v2', 'k3', 'v3')
print(map_col)

Column<'map(k1, v1, k2, v2, k3, v3)'>


In [6]:
df = spark.createDataFrame(
        [(1, 'aa', 'x', 0.5, 1.0, '_'), 
         (2, 'bb', 'y', 0.6, 2.0, '$'), 
         (3, 'cc', 'z', 0.7, 3.0, '^')], \
         "k1: int, v1: string, k2: string, v2: float, k3: float, v3: string")

df2 = df.withColumn('map_data', map_col)
df2.show(truncate=False)

+---+---+---+---+---+---+-----------------------------+
|k1 |v1 |k2 |v2 |k3 |v3 |map_data                     |
+---+---+---+---+---+---+-----------------------------+
|1  |aa |x  |0.5|1.0|_  |{1 -> aa, x -> 0.5, 1.0 -> _}|
|2  |bb |y  |0.6|2.0|$  |{2 -> bb, y -> 0.6, 2.0 -> $}|
|3  |cc |z  |0.7|3.0|^  |{3 -> cc, z -> 0.7, 3.0 -> ^}|
+---+---+---+---+---+---+-----------------------------+



In [7]:
# df = spark.createDataFrame(
#         [(1, 'aa', '1', 0.5, 1.0, '_'), 
#          (2, 'bb', 'y', 0.6, 2.0, '$'), 
#          (3, 'cc', 'z', 0.7, 3.0, '^')], \
#          "k1: int, v1: string, k2: string, v2: float, k3: float, v3: string")

df2 = df.withColumn('map_data', map_col)
# df2.show(truncate=False)
df2.printSchema()

root
 |-- k1: integer (nullable = true)
 |-- v1: string (nullable = true)
 |-- k2: string (nullable = true)
 |-- v2: float (nullable = true)
 |-- k3: float (nullable = true)
 |-- v3: string (nullable = true)
 |-- map_data: map (nullable = false)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



In [8]:
from pyspark.sql.functions import expr, lit

map_col = create_map(
    'k1', expr("case when k1%2 = 0 then 'even' else ' odd' end"),
    lit('literal_k2'), 'v2',)

df3 = df.withColumn('map_data', map_col)
df3.show(truncate=False)

+---+---+---+---+---+---+------------------------------+
|k1 |v1 |k2 |v2 |k3 |v3 |map_data                      |
+---+---+---+---+---+---+------------------------------+
|1  |aa |x  |0.5|1.0|_  |{1 ->  odd, literal_k2 -> 0.5}|
|2  |bb |y  |0.6|2.0|$  |{2 -> even, literal_k2 -> 0.6}|
|3  |cc |z  |0.7|3.0|^  |{3 ->  odd, literal_k2 -> 0.7}|
+---+---+---+---+---+---+------------------------------+



In [9]:
from pyspark.sql.functions import element_at
map_col = create_map('k1', 'v1', 'k2', 'v2', 'k3', 'v3')

df4 = df.withColumn('map_data', map_col)
df4 = df4.withColumn('val_by_key_1', element_at('map_data', '1'))\
         .withColumn('val_by_key_y', element_at('map_data', 'y'))\
         .withColumn('val_by_key_30', element_at('map_data', '3.0'))

df4.select('map_data', 'val_by_key_1', 'val_by_key_y', \
            'val_by_key_30').show(truncate=False)

+-----------------------------+------------+------------+-------------+
|map_data                     |val_by_key_1|val_by_key_y|val_by_key_30|
+-----------------------------+------------+------------+-------------+
|{1 -> aa, x -> 0.5, 1.0 -> _}|aa          |null        |null         |
|{2 -> bb, y -> 0.6, 2.0 -> $}|null        |0.6         |null         |
|{3 -> cc, z -> 0.7, 3.0 -> ^}|null        |null        |^            |
+-----------------------------+------------+------------+-------------+



In [14]:
from pyspark.sql.functions import map_contains_key
map_col = create_map('k1', 'v1', 'k2', 'v2', 'k3', 'v3')

df4 = df.withColumn('map_data', map_col)
df4 = df4.withColumn('contains_1', map_contains_key('map_data', '1'))\
         .withColumn('contains_Y', map_contains_key('map_data', 'Y'))\
         .withColumn('contains_30', map_contains_key('map_data', '3.0'))

df4.select('map_data', 'contains_1', 'contains_Y', \
            'contains_30').show(truncate=False)

+-----------------------------+----------+----------+-----------+
|map_data                     |contains_1|contains_Y|contains_30|
+-----------------------------+----------+----------+-----------+
|{1 -> aa, x -> 0.5, 1.0 -> _}|true      |false     |false      |
|{2 -> bb, y -> 0.6, 2.0 -> $}|false     |false     |false      |
|{3 -> cc, z -> 0.7, 3.0 -> ^}|false     |false     |true       |
+-----------------------------+----------+----------+-----------+



In [35]:
from pyspark.sql.functions import map_entries
df = spark.createDataFrame(
        [(1, 'aa', 'x', 0.5, 1.0, '_'), 
         (2, 'bb', 'y', 0.6, 2.0, '$'), 
         (3, 'cc', 'z', 0.7, 3.0, '^')], \
         "k1: int, v1: string, k2: string, v2: float, \
            k3: float, v3: string")

df5 = df.withColumn('map_data', map_col)\
        .withColumn('array_map_entries', map_entries('map_data'))
df5 = df5.select('map_data', 'array_map_entries')
df5.show(truncate=False)

+-----------------------------+-----------------------------+
|map_data                     |array_map_entries            |
+-----------------------------+-----------------------------+
|{1 -> aa, x -> 0.5, 1.0 -> _}|[{1, aa}, {x, 0.5}, {1.0, _}]|
|{2 -> bb, y -> 0.6, 2.0 -> $}|[{2, bb}, {y, 0.6}, {2.0, $}]|
|{3 -> cc, z -> 0.7, 3.0 -> ^}|[{3, cc}, {z, 0.7}, {3.0, ^}]|
+-----------------------------+-----------------------------+



In [42]:
from pyspark.sql.functions import map_from_entries
df = spark.createDataFrame(
        [(1, 'aa', 'x', 0.5, 1.0, '_'), 
         (2, 'bb', 'y', 0.6, 2.0, '$'), 
         (3, 'cc', 'z', 0.7, 3.0, '^')], \
         "k1: int, v1: string, k2: string, v2: float, k3: float, v3: string")

df6 = df.withColumn('map_data', map_col)\
        .withColumn('array_map_entries', map_entries('map_data'))\
        .withColumn('map_from_array', map_from_entries('array_map_entries'))
df6 = df6.select('map_data', 'array_map_entries', 'map_from_array')
df6.show(truncate=False)

+-----------------------------+-----------------------------+-----------------------------+
|map_data                     |array_map_entries            |map_from_array               |
+-----------------------------+-----------------------------+-----------------------------+
|{1 -> aa, x -> 0.5, 1.0 -> _}|[{1, aa}, {x, 0.5}, {1.0, _}]|{1 -> aa, x -> 0.5, 1.0 -> _}|
|{2 -> bb, y -> 0.6, 2.0 -> $}|[{2, bb}, {y, 0.6}, {2.0, $}]|{2 -> bb, y -> 0.6, 2.0 -> $}|
|{3 -> cc, z -> 0.7, 3.0 -> ^}|[{3, cc}, {z, 0.7}, {3.0, ^}]|{3 -> cc, z -> 0.7, 3.0 -> ^}|
+-----------------------------+-----------------------------+-----------------------------+



In [66]:
from pyspark.sql.functions import when

map_col1 = create_map(lit('k1'), lit('v1'))
map_col2 = create_map(lit('k1'), lit('v1'), lit('k2'), lit('v2'))
map_col3 = create_map(lit('k1'), lit('v1'), lit('k2'), lit('v2'), \
                      lit('k3'), lit('v3'))

df = spark.range(7) #creates DataFrame with column 'id'  
df7 = df.withColumn('map_data', when(df['id']%3 == 0, map_col1)\
                               .when(df['id']%3 == 1, map_col2)\
                               .when(df['id']%3 == 2, map_col3))
df7.show(truncate=False)

+---+------------------------------+
|id |map_data                      |
+---+------------------------------+
|0  |{k1 -> v1}                    |
|1  |{k1 -> v1, k2 -> v2}          |
|2  |{k1 -> v1, k2 -> v2, k3 -> v3}|
|3  |{k1 -> v1}                    |
|4  |{k1 -> v1, k2 -> v2}          |
|5  |{k1 -> v1, k2 -> v2, k3 -> v3}|
|6  |{k1 -> v1}                    |
+---+------------------------------+



In [70]:
df7.coalesce(1).write.save(path='map_to_json', format='json', mode='overwrite')

In [75]:
df = spark.range(3)

key_list = ['k' + str(i) for i in range(1,6)] # generate keys list list with python 
key_list = list(map(lambda x: lit(x), key_list))

val_list = [2**i for i in range(1,6)] # generate values list list with python as power of 2
val_list = list(map(lambda x: lit(x), val_list))

map_col = map_from_arrays(array(key_list), array(val_list))

df8 = df.withColumn('map_data', map_col)
df8.show(truncate=False)

+---+-----------------------------------------------+
|id |map_data                                       |
+---+-----------------------------------------------+
|0  |{k1 -> 2, k2 -> 4, k3 -> 8, k4 -> 16, k5 -> 32}|
|1  |{k1 -> 2, k2 -> 4, k3 -> 8, k4 -> 16, k5 -> 32}|
|2  |{k1 -> 2, k2 -> 4, k3 -> 8, k4 -> 16, k5 -> 32}|
+---+-----------------------------------------------+



In [81]:
df9 = df8.withColumn('keys', map_keys('map_data'))\
         .withColumn('values', map_values('map_data'))
df9.show(truncate=False)

+---+-----------------------------------------------+--------------------+-----------------+
|id |map_data                                       |keys                |values           |
+---+-----------------------------------------------+--------------------+-----------------+
|0  |{k1 -> 2, k2 -> 4, k3 -> 8, k4 -> 16, k5 -> 32}|[k1, k2, k3, k4, k5]|[2, 4, 8, 16, 32]|
|1  |{k1 -> 2, k2 -> 4, k3 -> 8, k4 -> 16, k5 -> 32}|[k1, k2, k3, k4, k5]|[2, 4, 8, 16, 32]|
|2  |{k1 -> 2, k2 -> 4, k3 -> 8, k4 -> 16, k5 -> 32}|[k1, k2, k3, k4, k5]|[2, 4, 8, 16, 32]|
+---+-----------------------------------------------+--------------------+-----------------+



In [97]:
spark.stop()